In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from projet3 import utils
from projet3 import ead

ModuleNotFoundError: No module named 'projet3'

# Qualités nutritionnelles d'un repas à destination des sportifs
Pour un sportif être capable d'analyser la qualité de son repas et connaitre les
proportions des différent marconutriments et/ou micronutriments.

## Pré-ambule
Depuis la source de données accessible à l'addresse https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv, nous allons réaliser les tâches suivantes:
1. Analyser la qualité de la données ; boulons, complétude.
2. Analyse statisque univarié.
3. Analyse statisque bivarié.
4. Exploitation des données dans le but d'extraire les proportions en terme de macronutriments.


### 1. Analyse qualitative de la donnée


Version

In [3]:
utils.print_version()

NameError: name 'utils' is not defined

In [4]:
# Donnée brut
data_df = ead.get_data()

#Info
# les 10 premiere lignes


NameError: name 'ead' is not defined

Extrait du dataset

In [ ]:
data_df.head(10)

Info complète du dataset

In [ ]:
data_df.info(verbose=True, null_counts=True)


Description

In [ ]:
data_df.describe()


Les lignes en doublons

In [ ]:
data_df[data_df.duplicated()==True]

Complétude des colonnes

In [ ]:
completude_df = ead.get_columns_completude()

In [ ]:
completude_gt_70 = completude_df[completude_df["Completude"] >= 70.0]

In [ ]:
completude_gt_70

In [ ]:
print(f'Dimension {completude_gt_70.shape}')


In [ ]:
completude_lt_001 = completude_df[completude_df["Completude"] < 0.01]

In [ ]:
completude_lt_001

In [ ]:
print(f"Dimension {completude_lt_001.shape}")


#### Résultat
Données issues de l'open data disponible gratuitement. Elle est composé de plus d'un million d'entrée et 175 colonnes:
- pour 51 colonnes on dispose de moins des 0.01% des lignes de renseignées, ces colonnes peuvent être omises de notre analyse
- pour 26 colonnes on dispose de plus de 70% des lignes de renseignées
- 14 lignes sont en doublons
- 119 colonnes correspondes à des données quantitative dont seulement 50 pour lesquels on dispose de plus de 1000 lignes (0.01% du volumes) de renseignées

#### Conclusion
Les premières analyses sont plutôt positives; on dispose de peu de doublons et on dispose de 26 colonnes pour lesquelles plus de 70% des lignes sont renseignées.


### 2. Analyse statisque univariée
Dans cette partie nous allons procéder à l'analyse univariée sur les colonnes retenues dans le cadre de notre analyse

#### Choix des colonnes
Dans un premier temps, sont retenues les colonnes pour lesquelles 70% de lignes sont remplies

In [ ]:
completude_gt_70["Column"]

Seules les colonnes
```
68                energy_100g
70                   fat_100g
71         saturated-fat_100g
106        carbohydrates_100g
107               sugars_100g
117             proteins_100g
121                 salt_100g
122               sodium_100g
```
Correspondes à des données quantitatives. Nous allons donc étendre notre analyse a certaines colonnes incluses dans

In [ ]:
completude_df[(completude_df["Completude"] >= 10.0)&(completude_df["Completude"] < 70.0)]

On retient alors les colonnes suivantes
```
104	trans-fat_100g	14.303652
105	cholesterol_100g	14.382140
116	fiber_100g	29.860238
124	vitamin-a_100g	13.779704
129	vitamin-c_100g	14.132801
143	calcium_100g	14.473610
145	iron_100g	14.142711
```
On regrettera de ne pas pouvoir retenir les vitamines B, leur couverture étant trop faible.

In [ ]:
all_columns = [
    'calcium_100g','cholesterol_100g', 'trans-fat_100g', 'iron_100g',
    'vitamin-c_100g', 'vitamin-a_100g', 'fat_100g', 'saturated-fat_100g', 
    'sugars_100g', 'carbohydrates_100g', 'proteins_100g','salt_100g',  
    'sodium_100g'
]
energie = ["energy_100g"]
categories_columns = ['code', 'product_name']

### Détection et traitement des données aberrantes.

In [ ]:
# Restrictions aux index étudiés
data_restricted_df = data_df[all_columns]
data_restricted_df.describe()

On constate des données < 0 et d'autre > 100g, on va donc procéder à des remplacements

In [ ]:
# TODO: remplacer par la mediane

for c in data_restricted_df.columns:
    data_restricted_df[c][data_restricted_df[c] <= 0] = data_restricted_df[c].median()
    data_restricted_df[data_restricted_df >= 100] = data_restricted_df[c].median()
    data_restricted_df[data_restricted_df.isnull()] = 0

In [ ]:
# Vérification
data_restricted_df.describe()

Visualistaion des boites à moustache

In [ ]:
n = len(data_restricted_df.columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))
for i, c in enumerate(data_restricted_df.columns):
    plt.sca(ax[i])
    try:
        ax_ = sns.boxplot(data=data_restricted_df[c])
        plt.title(c)
    except ValueError as error:
        print(error)

Résultats pour les colonnes, calcium_100g, cholesterol_100g,trans-fat_100g,iron_100g,vitamin-c_100g, vitamin-a_100g les données sont trop applaties, nous avons un problème d'échelle. 
Ces données seront traitées en mg pour 100g.

In [ ]:
data_restricted_df["calcium_100g"] *= 1000
data_restricted_df["cholesterol_100g"] *= 1000
data_restricted_df["trans-fat_100g"] *= 1000
data_restricted_df["vitamin-c_100g"] *= 1000
data_restricted_df["vitamin-a_100g"] *= 1000

Affichage des boites à moustache sans les outliers afin de pouvoir afficher les quartiles pour toutes les colonnes.

In [ ]:
n = len(data_restricted_df.columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))
for i, c in enumerate(data_restricted_df.columns):
    plt.sca(ax[i])
    try:
        y_values = data_restricted_df[c]
        # affichage sans les outilers
        ax_ = sns.boxplot(data=data_restricted_df[c], showfliers=False)
        plt.title(c)
        #plt.ylim(0, 110)
    except ValueError as error:
        print(error)

### Affichage des histogrammes pour les différents index

In [ ]:
# 
columns  = all_columns
n = len(columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))

for i, s in enumerate(columns):
    try:
        plt.sca(ax[i])
        print(f"Analyse de {s}")
        print(data_restricted_df[s].describe())
        sns.distplot(data_restricted_df[s], kde=True, norm_hist=True)
    except Exception as error:
        print(f'{error} while the display')


### Affichage repartition 

In [ ]:
categs = ["nova_group", "nutrition_grade_fr", "pnns_groups_2", "pnns_groups_1"]
n = len(categs)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))

for i, s in enumerate(categs):
    try:
        plt.sca(ax[i])
        # trop de temps freeze
        # sns.barplot(data_df[s])
    except Exception as error:
        print(f'{error} while the display')


### Résultats
#TODO : rajouter le fait que pour ce colonnes les valeurs ne sont pas très parlante; se rapprocher d"
Une fois de plus on constate que les colonnes calcium_100g, cholesterol_100g,trans-fat_100g,iron_100g,vitamin-c_100g, vitamin-a_100g sont très applaties.

### Conclusion
Etant donnée le cadre de notre projet; répartition des macronutriment et/ou micronutriments dans le cadre de d'une alimentation de sportif. 
Nous allons omettre ces colonnes afin de préciser notre projet et nous concentrer sur la répartition des macronutriments.


## 3. ANALYSE STATISTIQUE MULTIVARIE

### Affichage des pair plots

In [ ]:
sns.pairplot(data_df[all_columns])

### Affichage de la Heat map

In [ ]:
sns.heatmap(data_df[all_columns].select_dtypes(include=np.number).corr())

### Résultat
On constate une forte corrélation etre le sodium et le sel.


## 4. EXPLOITATION DES DONNEES
Analyse de la composition d'un repas, composé de plusiuers produits

In [ ]:
columns = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 
           'carbohydrates_100g', 'proteins_100g','salt_100g',  
           'calcium_100g', 'cholesterol_100g',
           'trans-fat_100g', 'iron_100g', 'vitamin-c_100g', 
           'vitamin-a_100g', 'alcohol_100g']
columns_mg = ["vitamin-a_100g", "vitamin-c_100g", "iron_100g"]

Realisationd d'un echantillon (repas) compose de 5 produits

In [ ]:
sample = data_df.sample(5)
codes = sample["code"]
sum_ = sample.sum()


Nom des produits 

In [ ]:
sample[["product_name"]]

Données de référence

In [ ]:
reference = utils.get_reference(data_df, columns, columns_mg)

In [ ]:
# Données de référence
reference

In [ ]:
# Données en Kcal
metrics_brut = utils.get_metrics(sample, codes=list(codes), columns=columns, columns_mg=columns_mg)
metrics_brut

Comparaison repas/reference

In [ ]:
sns.set_color_codes("pastel")
sns.set(style="whitegrid")
data_metrics = pd.DataFrame({"Repas": metrics_brut, "Reference": reference})
data_metrics.plot.barh()
plt.show()


# Affichage radar

In [ ]:
utils.plot_radar(metrics_brut, columns)
utils.plot_radar(reference, columns)


Repartition energétique

In [ ]:
sample_c = utils.get_kcal(metrics_brut)
sample_c.describe()

Reparition energetique du repas

In [ ]:
sample_c.plot.pie(label="repas", autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2, figsize=(18, 9))

Reparition energetique de la référence

In [ ]:
sample_ref_c = utils.get_kcal(reference)
sample_ref_c.plot.pie(label="repas", autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2, figsize=(18, 9))

### 5. CONCLUSION 
Dans la dernière partie nous nous sommes concentré à calculer la répartition des macronutiments pour 100g de produit, 

In [ ]:
# TODO:
camembert plus grand 
energie attention effet de sueil
reference statique 